In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression

In [5]:
a2 = "Business_ModeloRecomendacion.csv"

In [6]:
Business = pd.read_csv(a2)

In [7]:
Business

,business_id,name,address,latitude,longitude,stars,review_count,state
0,0x89c6c63c8cd87141:0x54d0d283872eecbb,Kater Street Bicycle,"Kater Street Bicycle, 609 S 16th St, Philadelp...",39.943673,-75.168762,4.9,15,PA
1,0x89c6c715d1821fe3:0x9cfa8308c0ce2289,Cadence Cycling and Cafe - Center City,"Cadence Cycling and Cafe - Center City, 201 S ...",39.950078,-75.180729,4.8,38,PA
2,0x89c6b0839da68acb:0xdefc7bf5c5a05cdc,Glenside Bible Church,"Glenside Bible Church, 447 N Keswick Ave, Glen...",40.110587,-75.149630,4.7,14,PA
3,0x89ce2ea137df5db7:0xab89dd7b3d947ac,Wellsboro Building Supply,"Wellsboro Building Supply, 45 Charleston St, W...",41.750531,-77.290827,4.6,28,PA
4,0x89ce2c1c3c5426df:0x1e132faacfd8005b,Charleston Street Recreation Area,"Charleston Street Recreation Area, 91 Charlest...",41.749456,-77.281806,4.6,14,PA
...,...,...,...,...,...,...,...,...
102685,x_2IrYgFiQn7GOTTgWRbAw,The Vac & Sew Center,"200 Haddonfield Berlin Rd, Ste 5",39.857700,-74.987230,4.0,5,PA
102686,fn3ybdsRSrIDpKZTsRuAWg,INSPcenter/Thai Clinical Massage,"2625 N Meridian St, Unit 50",39.805057,-86.155975,4.0,12,IN
102687,2O2K6SXPWv56amqxCECd4w,The Plum Pit,4405 Pennell Rd,39.856185,-75.427725,4.5,14,PA
102688,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,53.468419,-113.492054,3.0,13,IN


In [8]:
# Eliminar las columnas especificadas
Business = Business.drop(['address', 'latitude', 'longitude'], axis=1)

In [9]:
Business.head()

,business_id,name,stars,review_count,state
0,0x89c6c63c8cd87141:0x54d0d283872eecbb,Kater Street Bicycle,4.9,15,PA
1,0x89c6c715d1821fe3:0x9cfa8308c0ce2289,Cadence Cycling and Cafe - Center City,4.8,38,PA
2,0x89c6b0839da68acb:0xdefc7bf5c5a05cdc,Glenside Bible Church,4.7,14,PA
3,0x89ce2ea137df5db7:0xab89dd7b3d947ac,Wellsboro Building Supply,4.6,28,PA
4,0x89ce2c1c3c5426df:0x1e132faacfd8005b,Charleston Street Recreation Area,4.6,14,PA


In [10]:
numeric_business = Business.select_dtypes(include=[np.number])
correlation_matrix = numeric_business.corr()
correlation_matrix

,stars,review_count
stars,1.000000,0.059768
review_count,0.059768,1.000000


In [11]:
# Selección de características relevantes
features = ['stars', 'review_count', 'state']

# Variable objetivo: Definir qué significa ser "exitoso"
# Por ejemplo, si tiene más de 4.0 estrellas y más de 50 reseñas
Business['exitoso'] = ((Business['stars'] > 3.0) & (Business['review_count'] > 50)).astype(int)

# Mostrar la distribución de la variable objetivo
print(Business['exitoso'].value_counts())

# Seleccionar las características para el modelo
X = Business[features]

# Variable objetivo
y = Business['exitoso']

# Codificar la columna 'state' con OneHotEncoding
X = pd.get_dummies(X, columns=['state'], drop_first=True)

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear un modelo de regresión logística
modelo_logistico = LogisticRegression()

# Entrenar el modelo con los datos de entrenamiento
modelo_logistico.fit(X_train_scaled, y_train)

# Predecir con los datos de prueba
y_pred = modelo_logistico.predict(X_test_scaled)

# Evaluar precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy}")

# Mostrar métricas de clasificación
print(classification_report(y_test, y_pred))

exitoso
0    86444
1    16246
Name: count, dtype: int64
Precisión del modelo: 0.9418151718765215
              precision    recall  f1-score   support

           0       0.95      0.99      0.97     17310
           1       0.92      0.69      0.79      3228

    accuracy                           0.94     20538
   macro avg       0.93      0.84      0.88     20538
weighted avg       0.94      0.94      0.94     20538

Exitoso


In [17]:
import joblib

# Exportar el modelo entrenado
joblib.dump(modelo_logistico, 'modelo_logistico.pkl')

# Exportar el escalador
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [16]:
def predecir_exito(stars, review_count, state):
    # Crear el DataFrame con la entrada
    entrada = pd.DataFrame({
        'stars': [stars],
        'review_count': [review_count],
        'state': [state]
    })
    
    # Codificar el estado con OneHotEncoding
    entrada = pd.get_dummies(entrada, columns=['state'], drop_first=True)
    
    # Asegurarse de que todas las columnas estén presentes
    for col in X.columns:
        if col not in entrada.columns:
            entrada[col] = 0
    
    # Reordenar las columnas para que coincidan con el modelo
    entrada = entrada[X.columns]
    
    # Escalar los datos de entrada
    entrada_scaled = scaler.transform(entrada)
    
    # Hacer la predicción
    prediccion = modelo_logistico.predict(entrada_scaled)
    
    # Devolver el resultado
    return "Exitoso" if prediccion[0] == 1 else "No Exitoso"

In [15]:
# Ejemplo de uso de la función de predicción para un negocio específico
resultado = predecir_exito(3.5, 112, 'IN')
print(resultado)

Exitoso
